In [1]:
import spacy
from spacy.tokens import Doc
from spacy.tokens import DocBin
from spacy.vocab import Vocab
from tqdm import tqdm
from hazm import PeykareReader
from itertools import islice

2023-07-25 14:10:10.198406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 14:10:13.552854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
peykare = PeykareReader(root = '/repo/ebi/myPosTagger/resources/peykare/TextLabelData/',universal_pos=True)

In [4]:
len(list(peykare.sents()))

344742

In [9]:
corpus_size = 344736#len(list(peykare.sents()))

In [10]:
next(peykare.sents())

[('منبع', 'NOUN'),
 (':', 'PUNCT'),
 (')', 'PUNCT'),
 ('مجلة', 'NOUN,EZ'),
 ('سروش', 'NOUN,EZ'),
 ('هفتگی', 'ADJ'),
 ('،', 'PUNCT'),
 ('مصاحبه', 'NOUN'),
 ('با', 'ADP'),
 ('رئیس', 'NOUN,EZ'),
 ('دفتر', 'NOUN,EZ'),
 ('الجزیره', 'NOUN'),
 ('در', 'ADP'),
 ('تهران', 'NOUN'),
 ('،', 'PUNCT'),
 ('۱۳۸۰', 'NUM'),
 ('(', 'PUNCT')]

In [12]:
from sklearn.model_selection import train_test_split


train_sents, test_sents = train_test_split(list(islice(peykare.sents(), corpus_size))
                                           , test_size=0.1, random_state=10)



In [14]:
nlp = spacy.blank("fa")

# train_data = islice(peykare.sents(), int(corpus_size*0.95))


db = DocBin()
for sent in tqdm(train_sents):
    words = [word[0] for word in sent]
    tags = [word[1] for word in sent]
    doc = Doc(Vocab(strings=words), words = words)
    for d, tag in zip(doc, tags):
        d.tag_ = tag
    db.add(doc)
    
db.to_disk('../persian_spacy2/train_uni_randomState10.spacy')


100%|███████████████████████████████████████████████████████| 310262/310262 [00:45<00:00, 6894.48it/s]
tcmalloc: large alloc 1818607616 bytes == 0xbb8cc000 @ 


In [16]:
db = DocBin()
for sent in tqdm(test_sents):
    words = [word[0] for word in sent]
    tags = [word[1] for word in sent]
    doc = Doc(Vocab(strings=words), words = words)
    for d, tag in zip(doc, tags):
        d.tag_ = tag
    db.add(doc)
db.to_disk('../persian_spacy2/test_uni_randomState10.spacy')

100%|█████████████████████████████████████████████████████████| 34474/34474 [00:04<00:00, 6898.06it/s]


In [7]:
persian_nlp = spacy.load('output2/model-last/')

In [11]:
sent = 'در ارزش‌گذاری اسلامی و انسانی، ‌هیچ تفاوتی میان جنس زن و مرد نیست.'
for sent in islice(test_data, 10):
    doc = persian_nlp(' '.join([w for w, tag in sent]))
    for y, x in zip(sent, doc):
        if y[1] == x.tag_:
            print(1)
            print(y[0], y[1])
        else:
            print(y[0], f'[{y[1]}]', x.tag_) 
    print('====================')

1


In [14]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

persian_nlp = spacy.load('output2/model-best/')
test_data = islice(peykare.sents(), int(corpus_size*0.9) , corpus_size)
preds = []
ys = []

for sent in tqdm(test_data, total=int(corpus_size*0.1)):
    doc = persian_nlp(' '.join([w for w, tag in sent]))
    y = [tag for w, tag in sent]
    pred = [w.tag_ for w in doc]
    if len(y) == len(pred):
        ys += y
        preds += pred


print(classification_report(ys, preds))

print('Precision                                   : %.3f'%precision_score(ys, preds, average='weighted'))
print('Recall                                      : %.3f'%recall_score(ys, preds, average='weighted'))
print('F1-Score                                    : %.3f'%f1_score(ys, preds, average='weighted'))


34474it [32:48, 17.51it/s]                                                                           


              precision    recall  f1-score   support

         ADV       0.93      0.95      0.94     15492
        ADVe       0.95      0.96      0.96      1114
          AJ       0.93      0.95      0.94     57871
         AJe       0.91      0.90      0.90     14645
          CL       0.80      0.86      0.83      1349
        CONJ       1.00      1.00      1.00     74057
       CONJe       1.00      0.91      0.95        74
         DET       0.98      0.97      0.98     18171
        DETe       0.90      0.93      0.92      2165
         INT       0.96      0.87      0.92        31
           N       0.97      0.95      0.96    152057
         NUM       0.98      0.99      0.99     17964
        NUMe       0.89      0.93      0.91      1690
          Ne       0.97      0.97      0.97    119251
           P       1.00      1.00      1.00     70915
       POSTP       1.00      1.00      1.00     12911
         PRO       0.98      0.98      0.98     23117
        PROe       0.87    

In [8]:
from sklearn.metrics import accuracy_score
print('Accuracy                                   : %.3f'%accuracy_score(ys, preds))

Accuracy                                   : 0.975


In [7]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

persian_nlp = spacy.load('output2/model-best/')
test_data = islice(peykare.sents(), int(corpus_size*0.9) , corpus_size)
preds = []
ys = []

for sent in tqdm(test_data, total=int(corpus_size*0.1)):
    doc = persian_nlp(' '.join([w for w, tag in sent]))
    y = [tag for w, tag in sent]
    pred = [w.tag_ for w in doc]
    if len(y) == len(pred):
        ys += y
        preds += pred

    
print(classification_report(ys, preds))

print('Precision                                   : %.3f'%precision_score(ys, preds, average='macro'))
print('Recall                                      : %.3f'%recall_score(ys, preds, average='macro'))
print('F1-Score                                    : %.3f'%f1_score(ys, preds, average='macro'))


 76%|███████████████████████████████████████████████████████████████████████████████████▊                          | 26281/34473 [30:45<10:34, 12.91it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

34474it [38:09, 15.06it/s]                                                                                                                               


              precision    recall  f1-score   support

         ADV       0.93      0.95      0.94     15485
        ADVe       0.95      0.96      0.96      1114
          AJ       0.93      0.95      0.94     57871
         AJe       0.91      0.90      0.90     14645
          CL       0.80      0.86      0.83      1349
        CONJ       1.00      1.00      1.00     74055
       CONJe       1.00      0.91      0.95        74
         DET       0.98      0.97      0.98     18177
        DETe       0.90      0.93      0.92      2165
         INT       0.96      0.87      0.92        31
           N       0.97      0.95      0.96    152057
         NUM       0.98      0.99      0.99     17963
        NUMe       0.89      0.93      0.91      1690
          Ne       0.97      0.97      0.97    119251
           P       1.00      1.00      1.00     70919
       POSTP       1.00      1.00      1.00     12911
         PRO       0.98      0.98      0.98     23117
        PROe       0.87    

In [6]:
from hazm import *
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

test_data = islice(peykare.sents(), int(corpus_size*0.9) , corpus_size)
preds = []
ys = []
tagger = POSTagger(model='/home/sobhe/baaz/resources/postagger.model')

for sent in tqdm(test_data, total=int(corpus_size*0.1)):
    tagged = tagger.tag([w for w, tag in sent])
    y = [tag for w, tag in sent]
    pred = [tag for w, tag in tagged]
    if len(y) == len(pred):
        ys += y
        preds += pred
        
print(classification_report(ys, preds))

print('Precision                                   : %.3f'%precision_score(ys, preds, average='macro'))
print('Recall                                      : %.3f'%recall_score(ys, preds, average='macro'))
print('F1-Score                                    : %.3f'%f1_score(ys, preds, average='macro'))
    

34474it [01:12, 477.56it/s]                                                                                                                              


              precision    recall  f1-score   support

         ADV       0.97      0.96      0.97     18901
        ADVe       1.00      0.99      0.99      1376
          AJ       0.97      0.96      0.97     68208
         AJe       0.94      0.95      0.94     17071
          CL       0.93      0.94      0.93      1578
        CONJ       1.00      1.00      1.00     92009
       CONJe       1.00      1.00      1.00        96
         DET       0.98      0.97      0.98     22226
        DETe       0.97      0.97      0.97      2661
         INT       0.98      0.97      0.98        62
           N       0.98      0.97      0.97    182593
         NUM       0.99      0.99      0.99     20194
        NUMe       0.95      0.93      0.94      1877
          Ne       0.97      0.98      0.97    139753
           P       1.00      1.00      1.00     84876
       POSTP       1.00      1.00      1.00     16152
         PRO       0.99      0.99      0.99     29719
        PROe       0.96    

In [30]:
for i in range(3):
    items = next(y)
    words = [word[0] for word in items]
    tags = [word[1] for word in items]
    doc = Doc(Vocab(strings=words), words = words)
    for d, tag in zip(doc, tags):
        print(d.text, d.pos_)
        db.add(doc)

فکر 
می‌کنم 
بهترین 
مسئله 
درباره‌ی 
تنها 
کار 
کردن 
اینست 
که 
تصور 
شخصیت 
را 
دنبال 
می‌کنی 
، 
احساس 
خودت 
را 
دنبال 
می‌کنی 
و 
شخصیتها 
درست 
از 
تو 
هستند 
. 
به‌طورکلی 
روابط 
خوبی 
با 
همکاران 
نویسنده‌ام 
داشته‌ام 
. 
دوست 
دارم 
تنها 
بنویسم 
و 
اگر 
ایده‌ام 
با 
یک 
همکار 
جور 
باشد 
، 
با 
او 
به 
صورت 
مشترک 
بنویسم 
. 


In [7]:
# coding: utf-8

"""این ماژول شامل کلاس‌ها و توابعی برای خواندن پیکرهٔ Peykare است. 
[peykare پیکرهٔ](https://www.peykaregan.ir/dataset/%D9%BE%DB%8C%DA%A9%D8%B1%D9%87-%D9%85%D8%AA%D9%86%DB%8C-%D8%B2%D8%A8%D8%A7%D9%86-%D9%81%D8%A7%D8%B1%D8%B3%DB%8C)
جموعه‌ای از متون نوشتاری و گفتاری رسمی زبان فارسی است که از منابع واقعی همچون
روزنامه‌ها، سایت‌ها و مستنداتِ از قبل تایپ‌شده، جمع‌آوری شده، تصحیح گردیده و
برچسب خورده است. حجم این دادگان حدوداً ۱۰۰ میلیون کلمه است و از منابع مختلف تهیه
گردیده و دارای تنوع بسیار زیادی است. ۱۰ میلیون کلمه از این پیکره با استفاده از
۸۸۲ برچسب نحوی-معنایی به صورت دستی توسط دانشجویان رشتهٔ زبان‌شناسی برچسب‌دهی
شده‌اند و هر پرونده بر حسب موضوع و منبع آن طبقه‌بندی شده است. این پیکره که توسط
پژوهشکده پردازش هوشمند علائم تهیه شده است، برای استفاده در آموزش مدل زبانی و
سایر پروژه‌های مربوط به پردازش زبان طبیعی مناسب است.
"""

from __future__ import unicode_literals
import os, codecs
from hazm import Normalizer
from hazm import WordTokenizer


def coarse_pos_u(tags, word):
    """برچسب‌های ریز را به برچسب‌های درشت منطبق با استاندارد جهانی (coarse-grained universal pos tags) تبدیل می‌کند.
    Examples:
            >>> coarse_pos_u(['N','COM','SING'], 'الجزیره')
            'NOUN'
    Args:
            tags (List[str]): لیست برچسب‌های ریز.
    Returns:
            (List[str]): لیست برچسب‌های درشت جهانی.
    """

    map_pos_to_upos = {
        "N": "NOUN",
        "V": "VERB",
        "AJ": "ADJ",
        "ADV": "ADV",
        "PRO": "PRON",
        "DET": "DET",
        "P": "ADP",
        "POSTP": "ADP",
        "NUM": "NUM",
        "CONJ": "CCONJ",
        "PUNC": "PUNCT",
        "CL": "NOUN",
        "INT": "INTJ",
        "RES": "NOUN",
    }
    sconj_list = {
        "که",
        "تا",
        "گرچه",
        "اگرچه",
        "چرا",
        "زیرا",
        "اگر",
        "چون",
        "چراکه",
        "هرچند",
        "وگرنه",
        "چنانچه",
        "والا",
        "هرچه",
        "ولو",
        "مگر",
        "پس",
        "چو",
        "چه",
        "بنابراین",
        "وقتی",
        "والّا",
        "انگاری",
        "هرچندكه",
        "درنتيجه",
        "اگه",
        "ازآنجاكه",
        "گر",
        "وگر",
        "وقتيكه",
        "تااينكه",
        "زمانيكه",
    }
    num_adj_list = {
        "نخست",
        "دوم",
        "اول",
        "پنجم",
        "آخر",
        "يازدهم",
        "نهم",
        "چهارم",
        "ششم",
        "پانزدهم",
        "دوازدهم",
        "هشتم",
        "صدم",
        "هفتم",
        "هفدهم",
        "آخرين",
        "سيزدهم",
        "يكم",
        "بيستم",
        "ويكم",
        "دوسوم",
        "شانزدهم",
        "هجدهم",
        "چهاردهم",
        "ششصدم",
        "ميليونيم",
        "وهفتم",
        "يازدهمين",
        "هيجدهمين",
        "واپسين",
        "چهلم",
        "هزارم",
        "وپنجم",
        "هيجدهم",
        "ميلياردم",
        "ميليونيوم",
        "تريليونيوم",
        "چهارپنجم",
        "دهگانه",
        "ميليونم",
        "اوّل",
        "سوّم",
    }
    try:
        old_pos = list(
            set(tags)
            & {
                "N",
                "V",
                "AJ",
                "ADV",
                "PRO",
                "DET",
                "P",
                "POSTP",
                "NUM",
                "CONJ",
                "PUNC",
                "CL",
                "INT",
                "RES",
            }
        )[0] 
        if old_pos == "CONJ" and word in sconj_list:
            return "SCONJ"
        if old_pos == "NUM" and word in num_adj_list:
            return "ADJ" + (",EZ" if "EZ" in tags else "")
        return map_pos_to_upos[old_pos] + (",EZ" if "EZ" in tags else "")
    except:
        return "NOUN"


def coarse_pos_e(tags, word):
    """برچسب‌های ریز را به برچسب‌های درشت (coarse-grained pos tags) تبدیل می‌کند.
    Examples:
            >>> coarse_pos_e(['N','COM','SING'], 'الجزیره')
            'N'
    Args:
            tags (List[str]): لیست برچسب‌های ریز.
    Returns:
            (List[str]): لیست برچسب‌های درشت.
    """

    try:
        return list(
            set(tags)
            & {
                "N",
                "V",
                "AJ",
                "ADV",
                "PRO",
                "DET",
                "P",
                "POSTP",
                "NUM",
                "CONJ",
                "PUNC",
                "CL",
                "INT",
                "RES",
            }
        )[0] + (",EZ" if "EZ" in tags else "")
    except:
        return "N"


def join_verb_parts(sentence):
    """جمله را در قالب لیستی از `(توکن، برچسب)‌`ها می‌گیرد و توکن‌های مربوط به افعال چندبخشی را با کاراکتر زیرخط (_) به هم می‌چسباند.
    Examples:
            >>> join_verb_parts([('اولین', 'AJ'), ('سیاره', 'Ne'), ('خارج', 'AJ'), ('از', 'P'), ('منظومه', 'Ne'), ('شمسی', 'AJ'), ('دیده', 'AJ'), ('شد', 'V'), ('.', 'PUNC')])
            [('اولین', 'AJ'), ('سیاره', 'Ne'), ('خارج', 'AJ'), ('از', 'P'), ('منظومه', 'Ne'), ('شمسی', 'AJ'), ('دیده_شد', 'V'), ('.', 'PUNC')]
    Args:
            sentence(List[Tuple[str,str]]): جمله در قالب لیستی از `(توکن، برچسب)`ها.
    Returns:
            (List[Tuple[str, str]): لیستی از `(توکن، برچسب)`ها که در آن افعال چندبخشی در قالب یک توکن با کاراکتر زیرخط به هم چسبانده شده‌اند.
    """

    if not hasattr(join_verb_parts, "tokenizer"):
        join_verb_parts.tokenizer = WordTokenizer()
    before_verbs, after_verbs, verbe = (
        join_verb_parts.tokenizer.before_verbs,
        join_verb_parts.tokenizer.after_verbs,
        join_verb_parts.tokenizer.verbe,
    )

    result = [("", "")]
    for word in reversed(sentence):
        if word[0] in before_verbs or (
            result[-1][0] in after_verbs and word[0] in verbe
        ):
            result[-1] = (word[0] + "_" + result[-1][0], result[-1][1])
        else:
            result.append(word)
    return list(reversed(result[1:]))


class PeykareReader:
    """این کلاس شامل توابعی برای خواندن پیکرهٔ Peykare است.
    Args:
            root (str): آدرس فولدر حاوی فایل‌های پیکره.
            join_verb_parts (bool, optional): اگر `True‍` باشد افعال چندقسمتی به‌شکل چسبیده‌به‌هم برگردانده_می‌شود.
            pos_map (str): دیکشنری مبدل برچسب‌های ریز به درشت.
    """

    def __init__(
        self, root, joined_verb_parts=True, pos_map=coarse_pos_e, universal_pos=False
    ):
        self._root = root
        if pos_map is None:
            self._pos_map = lambda tags: ",".join(tags)
        elif universal_pos:
            self._pos_map = coarse_pos_u
        else:
            self._pos_map = coarse_pos_e
        self._joined_verb_parts = joined_verb_parts
        self._normalizer = Normalizer(punctuation_spacing=False, affix_spacing=False)

    def docs(self):
        """اسناد را به شکل متن خام برمی‌گرداند.
        Yields:
                (str): متن خام سند بعدی.
        """

        for root, dirs, files in os.walk(self._root):
            for name in sorted(files):
                with codecs.open(
                    os.path.join(root, name), encoding="windows-1256"
                ) as peykare_file:
                    text = peykare_file.read()
                    if text:
                        yield text

    def doc_to_sents(self, document):
        """سند ورودی را به لیستی از جملات تبدیل می‌کند.
        هر جمله لیستی از `(کلمه, برچسب)`ها است.
        Args:
                document (str): سندی که باید تبدیل شود.
        Yields:
                (List[(str,str)]): `ها جملهٔ بعدی در قالب لیستی از `(کلمه، برچسب).
        """

        sentence = []
        for line in document.split("\r\n"):
            if not line:
                continue

            parts = line.split(" ")
            tags, word = parts[3], self._normalizer.normalize("‌".join(parts[4:]))

            if word and word != "#":
                sentence.append((word, tags))

            if parts[2] == "PUNC" and word in {"#", ".", "؟", "!"}:
                if len(sentence) > 1:
                    yield sentence
                sentence = []

    def sents(self):
        """جملات پیکره را در قالب لیستی از `(توکن، برچسب)`ها برمی‌گرداند.
        Examples:
                >>> peykare = PeykareReader(root='corpora/peykare')
                >>> next(peykare.sents())
                [('دیرزمانی', 'N'), ('از', 'P'), ('راه‌اندازی', 'Ne'), ('شبکه‌ی', 'Ne'), ('خبر', 'Ne'), ('الجزیره', 'N'), ('نمی‌گذرد', 'V'), ('،', 'PUNC'), ('اما', 'CONJ'), ('این', 'DET'), ('شبکه‌ی', 'Ne'), ('خبری', 'AJe'), ('عربی', 'N'), ('بسیار', 'ADV'), ('سریع', 'ADV'), ('توانسته', 'V'), ('در', 'P'), ('میان', 'Ne'), ('شبکه‌های', 'Ne'), ('عظیم', 'AJe'), ('خبری', 'AJ'), ('و', 'CONJ'), ('بنگاه‌های', 'Ne'), ('چندرسانه‌ای', 'AJe'), ('دنیا', 'N'), ('خودی', 'N'), ('نشان', 'N'), ('دهد', 'V'), ('.', 'PUNC')]
                >>> peykare = PeykareReader(root='corpora/peykare', joined_verb_parts=False, pos_map=None)
                >>> next(peykare.sents())
                [('دیرزمانی', 'N,COM,SING,TIME,YA'), ('از', 'P'), ('راه‌اندازی', 'N,COM,SING,EZ'), ('شبکه‌ی', 'N,COM,SING,EZ'), ('خبر', 'N,COM,SING,EZ'), ('الجزیره', 'N,PR,SING'), ('نمی‌گذرد', 'V,PRES,NEG,3'), ('،', 'PUNC'), ('اما', 'CONJ'), ('این', 'DET,DEMO'), ('شبکه‌ی', 'N,COM,SING,EZ'), ('خبری', 'AJ,SIM,EZ'), ('عربی', 'N,PR,SING'), ('بسیار', 'ADV,INTSF,SIM'), ('سریع', 'ADV,GENR,SIM'), ('توانسته', 'V,PASTP'), ('در', 'P'), ('میان', 'N,COM,SING,EZ'), ('شبکه‌های', 'N,COM,PL,EZ'), ('عظیم', 'AJ,SIM,EZ'), ('خبری', 'AJ,SIM'), ('و', 'CONJ'), ('بنگاه‌های', 'N,COM,PL,EZ'), ('چندرسانه‌ای', 'AJ,SIM,EZ'), ('دنیا', 'N,COM,SING'), ('خودی', 'N,COM,SING,YA'), ('نشان', 'N,COM,SING'), ('دهد', 'V,SUB,POS,3'), ('.', 'PUNC')]
        Yields:
                (List[Tuple[str,str]]): جملهٔ بعدی در قالب لیستی از `(توکن، برچسب)`ها.
        """
        map_pos = lambda item: (item[0], self._pos_map(item[1].split(","), item[0]))

        for document in self.docs():
            for sentence in self.doc_to_sents(document):
                if self._joined_verb_parts:
                    sentence = join_verb_parts(sentence)
                yield list(map(map_pos, sentence))

In [20]:
peykare = PeykareReader(root = '/repo/ebi/myPosTagger/resources/peykare/TextLabelData/', universal_pos=True)

In [21]:
next(peykare.sents())

[('منبع', 'N,COM,SING'), (':', 'PUNC'), (')', 'PUNC'), ('مجلة', 'N,COM,SING,EZ'), ('سروش', 'N,PR,SING,EZ'), ('هفتگی', 'AJ,SIM'), ('،', 'PUNC'), ('مصاحبه', 'N,COM,SING'), ('با', 'P'), ('رئیس', 'N,COM,SING,EZ'), ('دفتر', 'N,COM,SING,LOC,EZ'), ('الجزیره', 'N,PR,SING'), ('در', 'P'), ('تهران', 'N,PR,SING,LOC'), ('،', 'PUNC'), ('۱۳۸۰', 'NUM,CAR,NOMI,SING'), ('(', 'PUNC')]


[('منبع', 'NOUN'),
 (':', 'PUNCT'),
 (')', 'PUNCT'),
 ('مجلة', 'NOUN,EZ'),
 ('سروش', 'NOUN,EZ'),
 ('هفتگی', 'ADJ'),
 ('،', 'PUNCT'),
 ('مصاحبه', 'NOUN'),
 ('با', 'ADP'),
 ('رئیس', 'NOUN,EZ'),
 ('دفتر', 'NOUN,EZ'),
 ('الجزیره', 'NOUN'),
 ('در', 'ADP'),
 ('تهران', 'NOUN'),
 ('،', 'PUNCT'),
 ('۱۳۸۰', 'NUM'),
 ('(', 'PUNCT')]